# A Simple FRP Implementation
In the last section, we have seen how to use a very simple API of signals and vars for functional reactive programming. In this unit, we are going to take a look under the covers and see how that simple API could be implemented.

We now develop a simple implementation of Signals and Vars, which
together make up the basis of our approach to functional reactive
programming.

## The Signal API

In [9]:
class Signal[T](expr: => T) {
def apply(): T = ???
}
object Signal {
def apply[T](expr: => T) = new Signal(expr)
}

defined class Signal
defined object Signal

## The Var API

In [10]:
class Var[T](expr: => T) extends Signal[T](expr) {
def update(expr: => T): Unit = ???
}
object Var {
def apply[T](expr: => T) = new Var(expr)
}

defined class Var
defined object Var

Implementation Idea:

Each signal maintains
* its current value,
* the current expression that defines the signal value,
* a set of observers: the other signals that depend on its value.

Then, if the signal changes, all observers need to be re-evaluated.

How do we record dependencies in observers?

* When evaluating a signal-valued expression, need to know which signal caller gets defined or updated by the expression.
* If we know that, then executing a `sig()` means adding caller to the observers of sig.
* When signal sig’s value changes, all previously observing signals are re-evaluated and the set `sig.observers` is cleared.
* Re-evaluation will re-enter a calling signal caller in `sig.observers`, as long as caller’s value still depends on `sig`.

How do we find out on whose behalf a signal expression is evaluated?

One simple (simplistic?) way to do this is to maintain a global data
structure referring to the current caller. (We will discuss and refine this
later).

That data structure is accessed in a stack-like fashion because one
evaluation of a signal might trigger others.

Here’s a class for stackable variables:

In [4]:
class StackableVariable[T](init: T) {
private var values: List[T] = List(init)
def value: T = values.head
def withValue[R](newValue: T)(op: => R): R = {
values = newValue :: values
try op finally values = values.tail
}
}

defined class StackableVariable

You access it like this
```scala

val caller = new StackableVar(initialSig)
caller.withValue(otherSig) { ... }
... caller.value ...
```

We also evaluate signal expressions at the top-level when there is no other
signal that’s defined or updated.
We use the “sentinel” 
object `NoSignal` as the caller for these expressions.


Together:
```scala
object NoSignal extends Signal[Nothing](???) { ... }
object Signal {
val caller = new StackableVariable[Signal[_]](NoSignal)
def apply[T](expr: => T) = new Signal(expr)
}
```

In [11]:
object NoSignal extends Signal[Nothing](???)
object Signal {
val caller = new StackableVariable[Signal[_]](NoSignal)
def apply[T](expr: => T) = new Signal(expr)
}

defined object NoSignal
defined object Signal

In [0]:
class Signal[T](expr: => T) {
import Signal._
private var myExpr: () => T = _
private var myValue: T = _
private var observers: Set[Signal[_]] = Set()
update(expr)
protected def update(expr: => T): Unit = {
myExpr = () => expr
computeValue()
}
protected def computeValue(): Unit = {
myValue = caller.withValue(this)(myExpr())
}
def apply() = {
observers += caller.value
assert(!caller.value.observers.contains(this), "cyclic signal definition")
myValue
}
}

cmd0.sc:2: not found: value Signal
import Signal._
       ^cmd0.sc:15: value += is not a member of Set[Helper.this.Signal[_]]
  Expression does not convert to assignment because:
    not found: value caller
    expansion: Signal.this.observers = Signal.this.observers.+(<caller: error>.<value: error>)
observers += caller.value
          ^cmd0.sc:16: not found: value caller
assert(!caller.value.observers.contains(this), "cyclic signal definition")
        ^cmd0.sc:12: not found: value caller
myValue = caller.withValue(this)(myExpr())
          ^Compilation Failed

: 